# 整个文档聚类；聚类数，阈值，权重选择；所有文档出现次数大于1的词训练词向量；加权平均计算质心；同一个簇中的文档中的每个词权重相加，作为该词的权重，提取四字词作为关键词

In [77]:
#!/usr/bin/python
#-*- encoding:utf-8 -*-
import os
import re
import numpy as np
from numpy import *
from sklearn.cluster import KMeans
from gensim.models import word2vec
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jieba
#jieba.load_userdict('/Users/lyj/Documents/项目/补充词典/word2.txt')
jieba.load_userdict('/Users/lyj/Desktop/提案程序/sogou.txt')
jieba.load_userdict('/Users/lyj/Desktop/提案程序/标题补充词典.txt')
import matplotlib.pyplot as plt
#from scipy.interpolate import spline
from sklearn import metrics

In [87]:
#读取stop停用词  
def readstopwords(stopwords_path): 
    stop_single_words=[]  
    with open(stopwords_path,'r+',encoding='utf-8') as lines:
        for line in lines:  
            content=line.strip()  
            stop_single_words.append(content) 
    return stop_single_words

#把文档中的所有文件名字存入数组
def getFilelist(path):
    txtNames = []                     #所有txt文档名字
    pathDir = os.listdir(path) #把文件夹里的所有txt文档的名字都获取了，放入数组
    for allDir in pathDir:
        if(allDir.find('txt') >= 0 and allDir.find('result') < 0):
            txtNames.append(allDir)
    return txtNames 

#具体的分词程序
def fenci(filePath,filename,stop_single_words) :
    f = open(filePath+'/'+filename,'r+',encoding='utf-8')
    file_list = f.read()
    f.close()
    #正则化必须去除字母数字百分数，不然计算的TF-IDF在用来求提案质心时会出错
    #错误类型：ValueError: 'GDP' is not in list
    file_list=re.sub('[A-Za-z0-9,\d+(\.\d+)?\%]','',file_list) #去除字母,数字,百分数
    #file_list=re.sub('[0-9,\d+(\.\d+)?\%]','',file_list)
    
    lis=[] #保存分词结果
    seg_list = jieba.cut(file_list,cut_all=False)
    for seg in seg_list : 
        seg = ''.join(seg.split())
        if seg not in stop_single_words and len(seg)!=1: #为什么计算质心的时候不能有单字词
            lis.append(seg)  
    return lis

#保存每篇提案的分词结果
def save_word_seg_result(filename,word_seg_result,save_word_seg_path):
    f = open(save_word_seg_path+"/"+filename,"w+",encoding='utf-8')
    f.write(' '.join(word_seg_result))
    f.close()
    return

#对标题分词
def title_Participle(filePath,save_word_seg_path,stop_single_words):   
    allfile = getFilelist(filePath) #获取所有提案的文件名
    for file in allfile:
        word_seg_result=fenci(filePath,file,stop_single_words)  #对每个提案分词
        save_word_seg_result(file,word_seg_result,save_word_seg_path) 
    return
        
#对整个文档分词
def Participle(filePath,save_word_seg_path,stop_single_words):   
    word_count={}    #存放所有提案的 词-数量 键值对
    reserved_words=[] #存放用于计算质心的词，也就是用于训练词向量的词
    doc_name_words={} #保存所有提案的 名字-分词结果 键值对
    sentences=[] #嵌套列表，保存每篇提案的分词结果
    allfile = getFilelist(filePath) #获取所有提案的文件名
    for file in allfile:
        word_seg_result=fenci(filePath,file,stop_single_words)  #对每个提案分词
        sentences.append(word_seg_result)
        sentences.append(word_seg_result)
        sentences.append(word_seg_result)
        sentences.append(word_seg_result)
        sentences.append(word_seg_result)
        for word in word_seg_result:  #统计所有提案中每个词的数量
            if word not in word_count.keys():
                word_count[word]=1
            else:
                word_count[word]+=1
        doc_name_words[file]=word_seg_result #将提案名和词按字典保存
    doc_name_words_new={}
    for key in doc_name_words.keys():
        word_list=[]
        for word_ in doc_name_words[key]:
            if word_count[word_]>1:
                word_list.append(word_)
        #保存分词结果，平时注释掉
        save_word_seg_result(key,word_list,save_word_seg_path) 
        doc_name_words_new[key]=word_list
    for word in word_count.keys(): 
        if word_count[word]>1:
            reserved_words.append(word)
    return doc_name_words_new,reserved_words,sentences,allfile

#训练词向量
def train_vec(sentences,dimen):
    model = word2vec.Word2Vec(sentences,min_count=2,size=dimen) #训练词向量
    return model

#读取800份已分词好的文档，进行TF-IDF计算。
#这里是去除停用词的所有词，且删除了出现次数小于某一阈值的词
def Tfidf(path,filelist):
    corpus = []  #存取800份文档的分词结果
    for ff in filelist :
        fname = path +"/"+ff
        f = open(fname,"r+",encoding="utf-8")
        content = f.read()
        corpus.append(content)
        f.close()

    vectorizer = CountVectorizer()    
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))
    tfidf_word = vectorizer.get_feature_names() #所有文本的分词词语
    tfidf_weight = tfidf.toarray()              #对应的tfidf矩阵
    return tfidf_word,tfidf_weight.tolist()

#计算每篇文档的质心
#doc_name_words中的词删除了停用词和小于某一阈值（出现次数小于1）的词
#reserved_words中的词删除了停用词和出现次数小于某一阈值的词
def centroid_TF(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,threshold,title_weight,title_seg_lis,dimen):
    doc_vec_list=[]
    doc_vec_dic={}
    for name in doc_name_words.keys():
        sum_value=np.mat(zeros((1,dimen)))
        count=0
        for word in doc_name_words[name]:
            weight=tfidf_weight[allfile.index(name)][tfidf_word.index(word)]
            if weight>=threshold:
                count+=1
                if word in title_seg_lis:
                    sum_value+=np.mat(model[word])*weight*title_weight#词向量*权重
                else:
                    sum_value+=np.mat(model[word])*weight #词向量*权重
            #print(tfidf_weight[allfile.index(name)][tfidf_word.index(word)])
        doc_vec_list.append(((sum_value/count).tolist())[0]) #这两处必须转化为列表，如果是数组，后面聚类程序保存文件的循环出错
        doc_vec_dic[name]=((sum_value/count).tolist())[0]
    return doc_vec_list,doc_vec_dic



In [88]:
#调参确定簇的个数
def cluster_num(doc_vec):
    
    radius_ave_list=[]
    diameter_ave_list=[]
    n_clusters_list=[]
    n_clusters_slope=[]
    contour_coefficient_list=[]
    
    for n_clusters in range(3,25):
        if n_clusters!=3:
            n_clusters_slope.append(n_clusters)
        n_clusters_list.append(n_clusters)
        radius_ave,diameter_ave=select_cluster_num(doc_vec,n_clusters)
        radius_ave_list.append(radius_ave)
        diameter_ave_list.append(diameter_ave)
        
        contour_coefficient=contour_coefficient_Calculation(doc_vec,n_clusters)
        contour_coefficient_list.append(contour_coefficient)
    
    #半径/直径的变化率
    radius_rate_change_list=[]
    diameter_rate_change_list=[]
    for i in range(len(n_clusters_list)-1):
        radius_rate_change=abs(radius_ave_list[i]-radius_ave_list[i+1])
        radius_rate_change_list.append(radius_rate_change)
        diameter_rate_change=abs(diameter_ave_list[i]-diameter_ave_list[i+1])
        diameter_rate_change_list.append(diameter_rate_change)
        
    draw_clusters_change(n_clusters_list,radius_ave_list,"The average radius varies with the number of clusters","average radius","the number of cluster")
    draw_clusters_change(n_clusters_list,diameter_ave_list,"The average diameter varies with the number of clusters","average diameter","the number of cluster")
    
    #draw_clusters_change_smooth(n_clusters_list,radius_ave_list,"The average radius varies with the number of clusters","average radius")
    #draw_clusters_change_smooth(n_clusters_list,diameter_ave_list,"The average diameter varies with the number of clusters","average diameter")
    
#     draw_clusters_change(n_clusters_slope,radius_rate_change_list,"Rate of change of average radius with number of clusters","Rate of change of average radius","the number of cluster")
#     draw_clusters_change(n_clusters_slope,diameter_rate_change_list,"Rate of change of average diameter with number of clusters","Rate of change of average diameter","the number of cluster")
    
    draw_clusters_change(n_clusters_list,contour_coefficient_list,"The contour coefficient varies with the number of clusters","contour coefficient","the number of cluster")
    
    #return n_clusters_list,radius_ave_list,diameter_ave_list


In [89]:
#计算每个簇的平均半径和平均直径
def select_cluster_num(doc_vec,n_clusters):
    clustering_result={}
    
    estimator = KMeans(n_clusters)#构造聚类器
    estimator.fit(doc_vec)#聚类
    label_pred = estimator.labels_ #获取聚类标签    
    
    #将每一类放入对应的列表中
    for i in range(len(label_pred)): #对于每一个聚类标签
        for j in range(n_clusters): #判断属于哪一类
            if label_pred[i]==j:
                if j in clustering_result.keys():
                    clustering_result[j].append(doc_vec[i])
                else:
                    clustering_result[j]=[doc_vec[i]] #键clustering_result[j]的值是一个嵌套列表
            else:
                pass
    
    #计算簇的平均半径
    radius_sum=0     #所有簇的半径和初始化
    for name in clustering_result.keys():
        radius_init=-1   #簇的半径初始化
        init_vec=np.mat(zeros([len(clustering_result[name][0])]))
        for vec in clustering_result[name]:
            init_vec+=np.mat(vec)
        centroid=init_vec/len(clustering_result[name]) #计算每个簇的质心
        for vec in clustering_result[name]:
            dist = np.sqrt(np.sum(np.square(np.mat(vec)-centroid)))
            if dist>radius_init: #计算簇的半径
                radius_init=dist
        radius_sum+=radius_init  #所有簇的半径求和
    radius_ave=radius_sum/n_clusters #平均半径
    
    #计算簇的平均直径
    diameter_sum=0
    for name in clustering_result.keys():
        diameter_init=-1
        for vec1 in clustering_result[name]:
            for vec2 in clustering_result[name]:
                dist = np.sqrt(np.sum(np.square(np.mat(vec1)-np.mat(vec2))))
                if dist>diameter_init:
                    diameter_init=dist
        diameter_sum+=diameter_init
    diameter_ave=diameter_sum/n_clusters
    
    return radius_ave,diameter_ave

#聚类
def kmeans_cluster(doc_vec_list,doc_vec_dic,n_clusters):
    clustering_result={}
    
    estimator = KMeans(n_clusters)#构造聚类器
    estimator.fit(doc_vec_list)#聚类
    label_pred = estimator.labels_ #获取聚类标签    
    
    name_list=[]
    vec_list=[]
    for key,value in doc_vec_dic.items( ):
        name_list.append(key)
        vec_list.append(value)
        
    #将每一类放入对应的列表中
    for i in range(len(label_pred)): #对于每一个聚类标签
        for j in range(n_clusters): #判断属于哪一类
            if label_pred[i]==j:
                if j in clustering_result.keys():
                    clustering_result[j].append(name_list[vec_list.index(doc_vec_list[i])])
                else:
                    clustering_result[j]=[name_list[vec_list.index(doc_vec_list[i])]] #键clustering_result[j]的值是一个嵌套列表
            else:
                pass
    return clustering_result

#将每个簇对应的文档保存到相应的文件夹下
def doc_save(clustering_result,filePath,savepath,n_clusters):

    for i in os.listdir(savepath):
        if i!='.DS_Store':
            path_file = os.path.join(savepath,i)  #取文件路径
            if os.path.isfile(path_file):
                os.remove(path_file)
            for f in os.listdir(path_file): 
                if f!='.DS_Store':
                    path_file2 =os.path.join(path_file,f)
                    if os.path.isfile(path_file2):
                        os.remove(path_file2)
                    
    for i in clustering_result.keys():
        for j in clustering_result[i]:
            f1 = open(filePath+"/"+j,"r+",encoding="utf-8")
            file_list = f1.read()
            f1.close()
            f2 = open(savepath+"/"+str(i+1)+".txt"+"/"+j,"w+",encoding="utf-8")
            f2.write(file_list)
            f2.close()
    #输出每一类的提案个数
    for i in range(n_clusters):
        print("主题"+str(i+1)+"的提案个数：",len(clustering_result[i]))
    return


In [90]:
#计算轮廓系数
def contour_coefficient_Calculation(doc_vec,n_clusters):
    clustering_result={}
    
    estimator = KMeans(n_clusters)#构造聚类器
    estimator.fit(doc_vec)#聚类
    label_pred = estimator.labels_ #获取聚类标签   
    
    #计算簇的轮廓系数
    contour_coefficient=metrics.silhouette_score(doc_vec, label_pred, metric='euclidean')
    
    return contour_coefficient

In [91]:
#调参确定保留多少词求每篇提案的质心
def centroid_word(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,title_seg_lis,dimen):
    contour_coefficient_list=[] #轮廓系数列表
    words_Threshold=[0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
    for threshold in words_Threshold:
        doc_vec_list,doc_vec_dic=centroid_TF(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,threshold,8,title_seg_lis,dimen)
        contour_coefficient=contour_coefficient_Calculation(doc_vec_list,10)
        contour_coefficient_list.append(contour_coefficient)
        print(threshold)
    draw_clusters_change(words_Threshold,contour_coefficient_list,"Contour coefficient changes with the number of words retained","contour coefficient","the number of words retained")
    

In [92]:
#调参确定标题正文的权重比
def title_text(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,title_seg_lis,dimen):
    contour_coefficient_list=[] #轮廓系数列表
    weight_ratio=[1,2,3,4,5,6,7,8]
    for title_weight in weight_ratio:
        doc_vec_list,doc_vec_dic=centroid_TF(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,0.08,title_weight,title_seg_lis,dimen)
        contour_coefficient=contour_coefficient_Calculation(doc_vec_list,14)
        contour_coefficient_list.append(contour_coefficient)
        print(title_weight)
    draw_clusters_change(weight_ratio,contour_coefficient_list,"Contour coefficient changes with the weight ratio of the title and body","contour coefficient","Weight ratio of title and body")
    

In [93]:
def draw_clusters_change(n_clusters_list,evaluation_index,title,ylabel,xlabel):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(n_clusters_list, evaluation_index,'ro')
    ax.plot(n_clusters_list, evaluation_index)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.grid(True)
    plt.show()
    return

def draw_clusters_change_smooth(n_clusters_list,evaluation_index,title,ylabel,xlabel):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    n_clusters_list_new = np.linspace(min(n_clusters_list),max(n_clusters_list),300) #300 represents number of points to make between T.min and T.max
    evaluation_index_smooth = spline(n_clusters_list,evaluation_index,n_clusters_list_new)
    ax.plot(n_clusters_list_new, evaluation_index_smooth,'r')
    ax.plot(n_clusters_list_new, evaluation_index_smooth)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.grid(True)
    plt.show()
    return

In [94]:
#提取聚类结果中同一个簇下的文档的关键词，将同一个词的TF-IDF相加，作为这个词的权重，选取权重最高的词作为这个簇的关键词
def key_word_cluster_TF_merge(savepath,key_words_savepath,tfidf_word,tfidf_weight,allfile):
    basic_keyword_num=3
    dic={}
    #循环savepath下文件夹中的每个子文件夹
    for i in os.listdir(savepath):
        if i!='.DS_Store':
            path_file = os.path.join(savepath,i)  #取子文件夹路径
            
            word_top=[] #存放该簇下的关键词
            word_weight={} #将该簇中的同一个词按照权重加起来
            
            #将同一个簇中的文档关键词按照TF-IDF取前keynum个，并存入word_top列表中
            cluster_files=getFilelist(path_file)
            for file in cluster_files:
                weight_index=allfile.index(file) #该篇文档所在的weight行
                for j in range(len(tfidf_word)):
                    if len(tfidf_word[j])>1:
                        if tfidf_word[j]!='北京' and tfidf_word[j]!='北京市':
                            if tfidf_word[j] not in word_weight.keys():
                                word_weight[tfidf_word[j]]=tfidf_weight[weight_index][j]
                            else:
                                word_weight[tfidf_word[j]]+=tfidf_weight[weight_index][j]
            word_weight=sorted(word_weight.items(),key=lambda A:A[1],reverse=True)
            keyword_num=basic_keyword_num*len(cluster_files)
            for k in word_weight[:keyword_num]: #取该簇中权重最大的前keynum个词
                word_top.append(k[0])
            if i=='1.txt':
                dic[1]=word_top[:3]
            if i=='2.txt':
                dic[2]=word_top[:3]
            if i=='3.txt':
                dic[3]=word_top[:3]
            if i=='4.txt':
                dic[4]=word_top[:3]
            if i=='5.txt':
                dic[5]=word_top[:3]
            if i=='6.txt':
                dic[6]=word_top[:3]
            if i=='7.txt':
                dic[7]=word_top[:3]
            if i=='8.txt':
                dic[8]=word_top[:3]
            #print(str(i),"  ",word_top[:3])
            
            f = open(key_words_savepath+"/"+str(i),"w+",encoding="utf-8")
            f.write(" ".join(word_top))
            f.close()
            
    for iii in sorted(dic.items(),key = lambda x:x[0]):
        print("主题"+str(iii[0])+"的关键词:",iii[1])
            

In [95]:
if __name__ == "__main__" : 
    stopwords="/Users/lyj/Desktop/提案程序/百度停用词列表（去除英文）+tycb_symbol.txt"  
    filePath = "/Users/lyj/Desktop/提案程序/提案数据"  #txt文本集所在的文件夹路径
    save_word_seg_path="/Users/lyj/Desktop/提案程序/zxtafenci" #保存分词结果的路径
    title_path="/Users/lyj/Desktop/提案程序" #所有标题所在的路径（所有标题在一个txt文档中）
    savepath="/Users/lyj/Desktop/提案程序/word2veccluster" #保存聚类结果
    title_save_word_seg_path='/Users/lyj/Desktop/提案程序/标题分词' #保存标题的分词结果
    key_words_savepath="/Users/lyj/Desktop/提案程序/候选关键词"

    stop_single_words=readstopwords(stopwords)
    title_seg_lis=list(set(fenci(title_path,'title.txt',stop_single_words))) #对标题分词 
    doc_name_words,reserved_words,sentences,allfile=Participle(filePath,save_word_seg_path,stop_single_words) #对整个文档分词
    tfidf_word,tfidf_weight=Tfidf(save_word_seg_path,allfile)
    
    dimen=300 #词向量维度
    model=train_vec(sentences,dimen)
    #求每篇文档的质心
    #这是一条说明语句doc_vec_list,doc_vec_dic=centroid_TF(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,threshold,weight,title_seg_lis)
    doc_vec_list,doc_vec_dic=centroid_TF(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,0.04,5,title_seg_lis,dimen)
    
    #调参确定各个参数
    #cluster_num(doc_vec_list) #确定簇的个数
    #centroid_word(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,title_seg_lis,dimen) #确定保留的词数
    #title_text(doc_name_words,reserved_words,model,allfile,tfidf_word,tfidf_weight,title_seg_lis,dimen) #确定标题-正文权重比
    
    
    #聚类，并将聚类结果保存，计算轮廓系数
    clustering_result=kmeans_cluster(doc_vec_list,doc_vec_dic,8)
    doc_save(clustering_result,filePath,savepath,8)
    contour_coefficient=contour_coefficient_Calculation(doc_vec_list,8)
    #print(contour_coefficient)
        
    #将聚类结果中，同一个簇中的文档中的同一个词的TF-IDF值相加提取关键词
    key_word_cluster_TF_merge(savepath,key_words_savepath,tfidf_word,tfidf_weight,allfile)
    #print("----------------------------------------------------------------------------------------------------------")
    
    

/Applications/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:123: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Applications/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:125: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


主题1的提案个数： 65
主题2的提案个数： 83
主题3的提案个数： 406
主题4的提案个数： 59
主题5的提案个数： 100
主题6的提案个数： 15
主题7的提案个数： 67
主题8的提案个数： 3
主题1的关键词: ['文化', '保护', '文化遗产']
主题2的关键词: ['产业', '企业', '创新']
主题3的关键词: ['建设', '城市', '发展']
主题4的关键词: ['教育', '教师', '学生']
主题5的关键词: ['停车', '车辆', '机动车']
主题6的关键词: ['垃圾', '垃圾分类', '分类']
主题7的关键词: ['服务', '社区', '医疗']
主题8的关键词: ['污泥', '资源化', '处理']
